In [ ]:
import pandas as pd
# Cambia 'archivo.csv' por el nombre real de tus archivos
df = pd.read_csv('titulos.csv')
print(df.head())


In [ ]:
print(df.info())
print(df.columns)
print(df.describe(include='all'))
# Para ver mejor la salida usar display
from IPython.display import display
display(df.describe(include='all'))


In [ ]:
print(df.isnull().sum())


In [ ]:
import matplotlib.pyplot as plt

df['tipo_institucion'].value_counts().plot(kind='barh')
plt.xlabel("Cantidad")
plt.ylabel("Tipo de institución")
plt.title("Cantidad por tipo de institución")
plt.tight_layout()
plt.show()



**Detección temprana de posibles duplicados semánticos en campos categóricos**

No existe un comando de pandas que "adivine" y te agrupe automáticamente las variantes, pero sí hay buenas prácticas y algunos trucos útiles:

1. Ver valores únicos en minúsculas y comparar contra los originales

In [ ]:
orig = set(df['tipo_institucion'].unique())
normalized = set(df['tipo_institucion'].str.lower().str.strip().unique())
print(f"Número de categorías originales: {len(orig)}")
print(f"Número de categorías normalizadas: {len(normalized)}")
if len(orig) != len(normalized):
    print("¡Atención: hay posibles duplicados por diferencias de mayúsculas/minúsculas o espacios!")


2. Mostrar “grupos” de valores que solo difieren por casing/espacios.
Esto te muestra exactamente qué valores únicos serían agrupados si tratas todo igual (útil para "ver" los falsos duplicados).

In [ ]:
# Crea un mapeo de valores originales agrupados por su versión normalizada
from collections import defaultdict
groups = defaultdict(set)
for val in df['tipo_institucion'].unique():
    groups[str(val).strip().lower()].add(val)
# Filtra los que tienen más de una variante:
duplicados = {norm: variants for norm, variants in groups.items() if len(variants) > 1}
print(duplicados)


3. Limpiar y ver Estadísticas rápidas:

In [ ]:
df['tipo_institucion_normalizado'] = df['tipo_institucion'].str.lower().str.strip()
print(df['tipo_institucion_normalizado'].value_counts())


In [ ]:
df['tipo_institucion_normalizado'].value_counts().plot(kind='barh')
plt.xlabel("Cantidad")
plt.ylabel("Tipo de institución")
plt.title("Cantidad por tipo de institución normalizado")
plt.tight_layout()
plt.show()



*Función para detectar posibles duplicados en todas las columnas*

In [ ]:
def detectar_duplicados_categoricos(df):
    """
    Detecta posibles duplicados por diferencias de mayúsculas/minúsculas 
    y espacios en todas las columnas categóricas del DataFrame
    """
    print("=== ANÁLISIS DE DUPLICADOS CATEGÓRICOS ===\n")
    
    # Filtrar solo columnas de tipo object (categóricas/string)
    columnas_categoricas = df.select_dtypes(include=['object']).columns
    
    problemas_encontrados = []
    
    for columna in columnas_categoricas:
        print(f"📊 Analizando: {columna}")
        
        # Obtener valores únicos originales y normalizados
        orig = set(df[columna].dropna().unique())
        # Normalización extendida: minúsculas + espacios + acentos
        normalized = set(
            df[columna]
            .dropna()
            .astype(str)
            .str.lower()
            .str.strip()
            .str.normalize('NFKD')
            .str.encode('ascii', 'ignore')
            .str.decode('utf-8')
            .unique()
        )
        
        print(f"   Categorías originales: {len(orig)}")
        print(f"   Categorías normalizadas: {len(normalized)}")
        
        if len(orig) != len(normalized):
            diferencia = len(orig) - len(normalized)
            print(f"   ⚠️  ATENCIÓN: {diferencia} posibles duplicados detectados!")
            problemas_encontrados.append((columna, diferencia))
        else:
            print("   ✅ Sin duplicados detectados")
        
        print("-" * 50)
    
    # Resumen final
    if problemas_encontrados:
        print("\n🚨 RESUMEN DE PROBLEMAS ENCONTRADOS:")
        for columna, cantidad in problemas_encontrados:
            print(f"   • {columna}: {cantidad} duplicados potenciales")
    else:
        print("\n✅ No se encontraron duplicados en ninguna columna categórica")

# Ejecutar la función
detectar_duplicados_categoricos(df)


**Técnicas más avanzadas para detectar duplicados "semánticos" que van más allá de la normalización básica:**

Función "integradora avanzada" adaptada para recorrer todas las columnas categóricas del DataFrame y aplicar fuzzy matching, abreviación y clustering:

In [ ]:
from fuzzywuzzy import fuzz
import itertools
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
import time
import numpy as np
import pandas as pd

# -------------------- Utilidades --------------------

def _normalizar_valores(valores, quitar_acentos=True):
    s = pd.Series(valores, dtype="object").astype(str).str.lower().str.strip()
    if quitar_acentos:
        s = (s
             .str.normalize('NFKD')
             .str.encode('ascii', 'ignore')
             .str.decode('utf-8'))
    return s.tolist()

def _estimacion_tiempo_pairs(n, k_ref=120_000, base=0.0):
    """
    Estima tiempo en segundos para comparar ~nC2 pares.
    k_ref controla la escala (ajústalo según tu máquina).
    """
    pares = n * (n - 1) / 2
    return base + (pares / k_ref)

# -------------------- Técnicas --------------------

def detectar_similares_fuzzy(valores, umbral=85, max_pairs=None):
    similares = []
    valores_unicos = list(dict.fromkeys(valores))  # preserva orden
    combos = itertools.combinations(valores_unicos, 2)

    if max_pairs is not None:
        combos = itertools.islice(combos, max_pairs)

    for val1, val2 in combos:
        ratio = fuzz.ratio(str(val1), str(val2))
        if ratio >= umbral:
            similares.append((val1, val2, ratio))
    return similares

def detectar_abreviaciones(valores):
    abreviaciones = []
    valores_unicos = list(dict.fromkeys(valores))
    for val1, val2 in itertools.combinations(valores_unicos, 2):
        str1, str2 = str(val1).lower(), str(val2).lower()
        if str1 in str2 or str2 in str1:
            abreviaciones.append((val1, val2))
        iniciales1 = ''.join([w[0] for w in str1.split() if w])
        iniciales2 = ''.join([w[0] for w in str2.split() if w])
        if iniciales1 == str2 or iniciales2 == str1:
            abreviaciones.append((val1, val2))
    return abreviaciones

def agrupar_similares_automatico(valores, eps=0.3, ngram_range=(2,3)):
    vec = TfidfVectorizer(analyzer='char_wb', ngram_range=ngram_range)
    X = vec.fit_transform([str(v) for v in valores])
    clustering = DBSCAN(eps=eps, min_samples=2, metric='cosine')
    labels = clustering.fit_predict(X)
    grupos = defaultdict(list)
    for i, label in enumerate(labels):
        if label != -1:
            grupos[int(label)].append(valores[i])
    return grupos

# -------------------- Función integradora --------------------

def diagnostico_avanzado_duplicados_allcols(
    df,
    tecnicas=('fuzzy', 'abrev', 'cluster'),
    fuzzy_umbral=85,
    max_unicos_auto=250,          # por encima de esto, se considera "pesada"
    max_pairs_fuzzy=200_000,      # límite duro de pares a evaluar con fuzzy
    normalizar=True,              # normalización textual previa
    quitar_acentos=True,          # eliminar acentos en normalización
    top_resultados=3,             # cantidad de ejemplos para imprimir por técnica
    eps_cluster=0.3,              # parámetro de DBSCAN
    ngram_range_cluster=(2,3)     # n-gramas para TF-IDF de clustering
):
    """
    Diagnóstico avanzado de duplicados en TODAS las columnas categóricas.
    - Evita cálculos costosos en columnas con demasiados únicos (salta con aviso).
    - Estima tiempo y limita pares a evaluar para fuzzy.
    - Aplica normalización (lower/strip/acentos) de forma opcional.
    """
    columnas_cat = df.select_dtypes(include=['object']).columns
    print("=== DIAGNÓSTICO AVANZADO DE DUPLICADOS (todas las columnas categóricas) ===\n")

    pesadas = []
    for col in columnas_cat:
        vals = df[col].dropna().unique().tolist()
        n = len(vals)
        print(f"\n🔍 Columna: {col}  |  únicos: {n}")

        if normalizar:
            vals_norm = _normalizar_valores(vals, quitar_acentos=quitar_acentos)
        else:
            vals_norm = [str(v) for v in vals]

        # Estimación de costo
        t_est = _estimacion_tiempo_pairs(n)
        print(f"   ⏱️ Estimación rápida (fuzzy ~O(n²)): ~{t_est:.1f}s")

        # Salto automático si es muy pesada
        if n > max_unicos_auto:
            print(f"   ⏭️  Saltada: demasiados valores únicos (> {max_unicos_auto}).")
            pesadas.append((col, n))
            continue

        # Técnicas
        if 'fuzzy' in tecnicas:
            start = time.time()
            # Limitar pares si hace falta
            max_pairs = max_pairs_fuzzy
            resultados = detectar_similares_fuzzy(vals_norm, umbral=fuzzy_umbral, max_pairs=max_pairs)
            dur = time.time() - start
            if resultados:
                print(f"   📊 Fuzzy (≥{fuzzy_umbral}%) → {len(resultados)} pares  |  {dur:.1f}s")
                for v1, v2, sc in resultados[:top_resultados]:
                    print(f"      {sc}%: '{v1}' ≈ '{v2}'")
            else:
                print(f"   📊 Fuzzy: sin pares sobre el umbral  |  {dur:.1f}s")

        if 'abrev' in tecnicas:
            start = time.time()
            abrev = detectar_abreviaciones(vals_norm)
            dur = time.time() - start
            if abrev:
                print(f"   🔤 Abreviaciones/contenimientos → {len(abrev)} pares  |  {dur:.1f}s")
                for v1, v2 in abrev[:top_resultados]:
                    print(f"      '{v1}' ↔ '{v2}'")
            else:
                print(f"   🔤 Abreviaciones: no se detectaron  |  {dur:.1f}s")

        if 'cluster' in tecnicas:
            start = time.time()
            grupos = agrupar_similares_automatico(vals_norm, eps=eps_cluster, ngram_range=ngram_range_cluster)
            dur = time.time() - start
            if grupos:
                print(f"   🎯 Clustering (DBSCAN) → {len(grupos)} grupos  |  {dur:.1f}s")
                for i, (g, items) in enumerate(grupos.items()):
                    if i < top_resultados:
                        print(f"      Grupo {g}: {items}")
            else:
                print(f"   🎯 Clustering: no se formaron grupos  |  {dur:.1f}s")

        print("-" * 80)

    if pesadas:
        print("\n⚠️ Columnas saltadas por ser muy pesadas (ajusta max_unicos_auto si deseas incluirlas):")
        for col, n in pesadas:
            print(f"   • {col}: {n} únicos")

# Ejemplo de uso:
diagnostico_avanzado_duplicados_allcols(
     df,
     tecnicas=('fuzzy', 'abrev', 'cluster'),
     fuzzy_umbral=85,
     max_unicos_auto=1000,
     max_pairs_fuzzy=200_000,
     normalizar=True,
     quitar_acentos=True,
     top_resultados=3,
     eps_cluster=0.3,
     ngram_range_cluster=(2,3)
 )
""" Sugerencias:

Ajusta max_unicos_auto según tu hardware. 250–400 suele ser razonable.

Si quieres incluir “documento”, sube max_unicos_auto pero baja max_pairs_fuzzy para acotar tiempo.

La estimación es orientativa; puedes calibrar k_ref para tu equipo. """